# Notebook 3 : Analyse pour la Construction de la Couche Gold

## Objectif

Ce notebook vous guide dans l'**analyse des données de la couche silver** pour préparer la construction de la couche **gold** (analytics/BI/ML).

La couche gold contient des données transformées, nettoyées et optimisées pour l'analyse métier et la création de rapports/dashboards.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :

1. **Exécuté le notebook `2_[LOAD]_load_to_bigquery.ipynb`** pour avoir toutes les tables dans BigQuery (dataset `silver`)
2. **Fichier `.env` configuré** avec les variables d'environnement nécessaires
3. **Service Account** avec les permissions BigQuery (`BigQuery Data Viewer`, `BigQuery Job User`)
4. **Packages Python installés** : `google-cloud-bigquery`, `pandas`, etc.

## Structure du Notebook

Ce notebook contient **4 tâches principales** à réaliser :

1. **Tâche 1 : Analyser la Granularité** - Comprendre le niveau de détail de chaque table
2. **Tâche 2 : Identifier les Transformations** - Déterminer les transformations nécessaires pour la couche gold
3. **Tâche 3 : Identifier les Clés de Jointure** - Mapper les relations entre les tables
4. **Tâche 4 : Analyse métier Identifier les **KPIs** métiers - 

## Résultats Attendus

À la fin de ce notebook, vous devriez avoir :
- Une compréhension claire de la structure et de la granularité de chaque table
- Une liste des transformations à appliquer pour créer la couche gold
- Un schéma de jointures documenté

## Configuration et Connexion à BigQuery

Cette section configure l'environnement et établit la connexion avec BigQuery pour explorer les données de la couche silver.


In [1]:
# Standard library imports
import os
from pathlib import Path

# Third-party imports
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

# Configuration
load_dotenv()

ROOT = Path.cwd().parent
PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
DATASET_ID = "silver"

# Authentification
creds = service_account.Credentials.from_service_account_file(SA_PATH)
bq_client = bigquery.Client(project=PROJECT_ID, credentials=creds)

print(f"[OK] - Connecté au projet: {PROJECT_ID}")
print(f"[OK] - Dataset: {DATASET_ID}")


[OK] - Connecté au projet: univ-reims-sncf-forecast
[OK] - Dataset: silver


---

## Tâche 1 : Analyser la Granularité de Chaque Table

### Objectif

La **granularité** d'une table correspond au niveau de détail des données qu'elle contient. Comprendre la granularité est essentiel pour :
- Déterminer comment agréger les données
- Identifier les duplications potentielles
- Comprendre le niveau de détail nécessaire pour les analyses métier

### Instructions

Pour chaque table du dataset `silver`, vous devez :

1. **Lister les colonnes** et leurs types
2. **Identifier les clés primaires** ou les colonnes qui identifient de manière unique une ligne
3. **Déterminer la granularité** : à quel niveau de détail sont les données ?
   - Exemple : `fact_validations` pourrait être au niveau **jour × gare × type de titre**
4. **Compter les lignes** et estimer la taille des données
5. **Identifier les colonnes de dimension** (références vers d'autres tables)

### Exemple de Format de Réponse

```
Table: dim_gare
- Granularité: 1 ligne = 1 gare
- Clé primaire: id_gares
- Nombre de lignes: 1234
- Colonnes de dimension: aucune (table de dimension)
```


### À Compléter : Analyse de Granularité

**Tables de Dimension :**

1. `dim_gare`
2. `dim_ligne`
3. `dim_arret`
4. `dim_vacances_scolaires`
5. `dim_transporteur`

**Tables de Fait :**

6. `fact_validations_*` (toutes les tables de validation)

**Votre tâche :** Exécutez des requêtes SQL pour analyser chaque table et remplir le tableau ci-dessous.


In [2]:
# Exemple : Analyser la table dim_gare
query = f"""
Votre code ici
"""

df = bq_client.query(query).to_dataframe()
print("=== Analyse de dim_gare ===")
display(df)

# Afficher le schéma
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_gare")
print("\n=== Schéma ===")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type} ({field.mode})")


=== Analyse de dim_gare ===


/Users/admin/.pyenv/versions/3.11.14/envs/etl-gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,nb_lignes,nb_gares_uniques,nb_id_null
0,1234,1231,0



=== Schéma ===
  - geo_point_2d: GEOGRAPHY (NULLABLE)
  - geo_shape: GEOGRAPHY (NULLABLE)
  - id_gares: INTEGER (REQUIRED)
  - nom_gares: STRING (NULLABLE)
  - nom_so_gar: STRING (NULLABLE)
  - nom_su_gar: STRING (NULLABLE)
  - id_ref_zdc: INTEGER (NULLABLE)
  - nom_zdc: STRING (NULLABLE)
  - id_ref_zda: INTEGER (NULLABLE)
  - nom_zda: STRING (NULLABLE)
  - idrefliga: STRING (NULLABLE)
  - idrefligc: STRING (NULLABLE)
  - res_com: STRING (NULLABLE)
  - indice_lig: STRING (NULLABLE)
  - mode: STRING (NULLABLE)
  - tertrain: STRING (NULLABLE)
  - terrer: STRING (NULLABLE)
  - termetro: STRING (NULLABLE)
  - tertram: STRING (NULLABLE)
  - terval: STRING (NULLABLE)
  - exploitant: STRING (NULLABLE)
  - idf: INTEGER (NULLABLE)
  - principal: INTEGER (NULLABLE)
  - x: FLOAT (NULLABLE)
  - y: FLOAT (NULLABLE)
  - picto: STRING (NULLABLE)
  - nom_iv: STRING (NULLABLE)


---

## Tâche 2 : Identifier les Transformations Nécessaires

### Objectif

Identifier les **transformations** à appliquer aux données de la couche silver pour créer la couche gold optimisée pour l'analyse.

### Types de Transformations Possibles

1. **Nettoyage des données**
   - Suppression des doublons
   - Gestion des valeurs NULL
   - Normalisation des formats (dates, textes)

2. **Enrichissement**
   - Ajout de colonnes calculées
   - Jointures avec les tables de dimension
   - Ajout de catégories/segments

3. **Agrégation**
   - Regroupement par dimensions (jour, gare, ligne, etc.)
   - Calcul de métriques (somme, moyenne, comptage)
   - Création de tables pré-agrégées

### Instructions

Pour chaque table, identifiez :
1. **Les problèmes de qualité** à corriger
2. **Les transformations nécessaires** avec des exemples concrets
3. **Les colonnes à ajouter** (calculées ou issues de jointures)
4. **Les agrégations possibles** pour optimiser les requêtes


In [3]:
# Exemple 1 : Vérifier les doublons dans fact_validations
query = f"""
Votre code ici
"""

df_duplicates = bq_client.query(query).to_dataframe()
print("=== Exemple : Recherche de doublons ===")
display(df_duplicates)


=== Exemple : Recherche de doublons ===


/Users/admin/.pyenv/versions/3.11.14/envs/etl-gcp/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,JOUR,ID_ZDC,CATEGORIE_TITRE,nb_occurrences
0,2023-11-03,999999,Amethyste,2
1,2023-11-06,999999,Amethyste,2
2,2023-11-08,999999,Amethyste,2
3,2023-11-09,999999,Amethyste,2
4,2023-11-12,999999,Amethyste,2
5,2023-11-14,999999,Amethyste,2
6,2023-11-17,999999,Amethyste,2
7,2023-07-01,71379,Amethyste,2
8,2023-07-02,71379,Amethyste,2
9,2023-07-03,71379,Amethyste,2


---

## Tâche 3 : Identifier les Clés de Jointure Possibles

### Objectif

Identifier toutes les **relations possibles** entre les tables pour pouvoir créer des jointures dans la couche gold.

### Instructions

Pour chaque paire de tables, identifiez :

1. **Les colonnes de jointure** (clés étrangères)
2. **Le type de relation** (1-1, 1-N, N-N)
3. **La cardinalité** (combien de lignes de la table A correspondent à combien de lignes de la table B)
4. **Vérifier l'intégrité référentielle** (toutes les clés étrangères existent-elles dans la table de dimension ?)


---

## Tâche 4 : Analyse métier

Identifier les **KPIs** (Key Performance Indicators) à calculer :
- Nombre total de validations par période
- Répartition par type de titre
- Top 10 des gares les plus fréquentées
- Comparaison jour ouvrable vs weekend

